# Prior prob

In [188]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel
)

bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
causal_prior_model = AutoModelForCausalLM.from_pretrained("facebook/bart-large")
bart_model = AutoModel.from_pretrained('facebook/bart-large')

Some weights of the model checkpoint at facebook/bart-large were not used when initializing BartForCausalLM: ['encoder.layers.0.fc2.weight', 'encoder.layers.1.self_attn.out_proj.bias', 'encoder.layers.4.self_attn.out_proj.bias', 'encoder.layers.8.self_attn.q_proj.weight', 'encoder.layers.10.self_attn_layer_norm.weight', 'encoder.layers.0.final_layer_norm.bias', 'encoder.layers.4.fc2.bias', 'encoder.layers.4.final_layer_norm.bias', 'encoder.layers.7.fc1.bias', 'encoder.layers.4.self_attn.q_proj.weight', 'encoder.layers.9.fc1.weight', 'encoder.layers.5.self_attn.out_proj.bias', 'encoder.layers.6.self_attn.out_proj.weight', 'encoder.layers.1.fc1.bias', 'encoder.layers.5.self_attn.q_proj.bias', 'encoder.layers.4.self_attn.v_proj.weight', 'encoder.layers.10.fc2.weight', 'encoder.layers.10.self_attn_layer_norm.bias', 'encoder.layers.6.self_attn.k_proj.bias', 'encoder.layers.3.self_attn.out_proj.bias', 'encoder.layers.4.self_attn.q_proj.bias', 'encoder.layers.5.self_attn.q_proj.weight', 'enco

## Load ENTFA data

In [178]:
from src.data_utils import load_EntFA

data_entfa = load_EntFA("test")

In [184]:
from datasets import load_dataset
xsum_test = load_dataset("xsum")["test"]
xsum_test_by_id = {
    doc["id"]: doc for doc in xsum_test
}

# Verify that doc in dataset is equal to XSUM doc
print(
    "Is Xsum doc equal to enfa source?", xsum_test_by_id["37989821"]["document"] == data_entfa[0]["source"]
)
print(
    "...with replaced newlines?", xsum_test_by_id["37989821"]["document"].replace("\n", " ") == data_entfa[0]["source"]
)

Using custom data configuration default
Reusing dataset xsum (/Users/anton164/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

Is Xsum doc equal to enfa source? False
...with replaced newlines? True


It's only equal if we replace newlines..

In [185]:
from src.data_utils import lookup_entfa_by_prediction

entfa_sydney = lookup_entfa_by_prediction(
    data_entfa,
    "Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed by a gunman in the Australian city."
)
entfa_sydney["prediction"], entfa_sydney["entities"]

('Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed by a gunman in the Australian city.',
 [{'start': 0,
   'end': 6,
   'label': 'Non-hallucinated',
   'type': 'ORG',
   'ent': 'Sydney'},
  {'start': 22,
   'end': 27,
   'label': 'Non-hallucinated',
   'type': 'ORDINAL',
   'ent': 'first'},
  {'start': 60,
   'end': 68,
   'label': 'Non-factual Hallucination',
   'type': 'ORG',
   'ent': 'Waverley'},
  {'start': 83,
   'end': 86,
   'label': 'Non-hallucinated',
   'type': 'CARDINAL',
   'ent': 'two'},
  {'start': 124,
   'end': 134,
   'label': 'Non-hallucinated',
   'type': 'NORP',
   'ent': 'Australian'}])

## Prior Prob

In [199]:
from src.masked_probability import prior_causal_probability
prior_causal_probability(
    causal_prior_model,
    bart_tokenizer,
    entfa_sydney["prediction"]
)

[('<s>', 0, 2.1549179541474617e-12),
 ('S', 104, 1.780824277375359e-06),
 ('yd', 9611, 2.7244109332968947e-06),
 ('ney', 2596, 3.707136642105979e-08),
 (' has', 34, 0.027413560077548027),
 (' marked', 4760, 1.1275883480266202e-06),
 (' the', 5, 0.00018941506277769804),
 (' first', 78, 0.0023676673881709576),
 (' anniversary', 4038, 5.814571068185614e-06),
 (' of', 9, 0.0017079596873372793),
 (' the', 5, 0.003059038193896413),
 (' siege', 19951, 3.6858385787930104e-10),
 (' at', 23, 0.0022077469620853662),
 (' the', 5, 0.0014361185021698475),
 (' W', 305, 0.002821172820404172),
 ('aver', 9903, 3.773652963445784e-07),
 ('ley', 607, 7.092088338822577e-08),
 (' cafe', 16381, 1.8298884185696807e-07),
 (' in', 11, 0.03579285740852356),
 (' which', 61, 2.48848186856776e-06),
 (' two', 80, 4.862476998823695e-05),
 (' women', 390, 0.00013294632663019001),
 (' were', 58, 0.00033202560734935105),
 (' killed', 848, 4.035213407860283e-08),
 (' by', 30, 4.279447966837324e-05),
 (' a', 10, 0.00526131

# Masked Prob

In [289]:
from transformers import AutoModelForMaskedLM
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
bart_masked = AutoModelForMaskedLM.from_pretrained("facebook/bart-large")

In [312]:
from src.masked_probability import prior_masked_probability
prior_masked_probability(
    bart_masked, 
    bart_tokenizer, 
    "Sydney has marked the first anniversary of the siege at the", 
    " Waverley"
) 

([{'masked_prob': tensor(0.0035),
   'input': '<s>Sydney has marked the first anniversary of the siege at the<mask>',
   'top_probs': [' Sydney',
    ' siege',
    ' Port',
    ' Siege',
    ' end',
    ' Australian',
    ' Hilton',
    ' NSW',
    ' Red',
    ' Royal']},
  {'masked_prob': tensor(1.7951e-05),
   'input': '<s>Sydney has marked the first anniversary of the siege at the W<mask>',
   'top_probs': [' Hotel',
    ' Building',
    ' Block',
    ' W',
    ' Bank',
    ' and',
    ' in',
    ' for',
    ' Club',
    ' on']},
  {'masked_prob': tensor(0.0003),
   'input': '<s>Sydney has marked the first anniversary of the siege at the Waver<mask>',
   'top_probs': [' Beach',
    ' Island',
    ' Creek',
    ' Point',
    ' Hill',
    ' House',
    ' Arms',
    ' Farm',
    ' Bay',
    ' Cove']}],
 tensor(2.0389e-11))

In [308]:
prior_masked_probability(
    bart_masked, 
    bart_tokenizer, 
    "Sydney has marked the", 
    " first"
) 

([{'masked_prob': tensor(0.0385),
   'input': '<s>Sydney has marked the<mask>',
   'top_probs': [' start',
    ' cent',
    ' end',
    ' 100',
    ' 50',
    ' first',
    ' 20',
    ' 10',
    ' one',
    ' 25']}],
 tensor(0.0385))

# Posterior Prob

In [65]:
from transformers import (
    AutoModelForSeq2SeqLM
)

bart_xsum = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

## What would bart-xsum have generated?

With and without replacing newline.

In [70]:
from src.generation_utils import generate_summaries
doc = xsum_test_by_id["35099282"]["document"]
docs_to_summarize = [
    doc, 
    doc.replace("\n", " ")
]

sums = generate_summaries(
    bart_xsum,
    bart_tokenizer,
    docs_to_summarize,
    None,
    num_beams=4
)
sums

['Sydney has marked the first anniversary of the siege at the Waverley cafe in which two people were killed.',
 'Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed.']

In [186]:
entfa_sydney["prediction"]

'Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed by a gunman in the Australian city.'

In [189]:
from src.masked_probability import forceful_conditional_generation
preds, sums = forceful_conditional_generation(
    bart_xsum,
    bart_tokenizer,
    entfa_sydney["prediction"],
    docs_to_summarize
)
preds

[('S', 104, tensor([0.1869, 0.2390])),
 ('yd', 9611, tensor([0.9439, 0.9353])),
 ('ney', 2596, tensor([0.9553, 0.9534])),
 (' has', 34, tensor([0.5095, 0.5292])),
 (' marked', 4760, tensor([0.3043, 0.2741])),
 (' the', 5, tensor([0.6830, 0.7088])),
 (' first', 78, tensor([0.6297, 0.6220])),
 (' anniversary', 4038, tensor([0.9380, 0.9416])),
 (' of', 9, tensor([0.8882, 0.8881])),
 (' the', 5, tensor([0.8174, 0.8127])),
 (' siege', 19951, tensor([0.3841, 0.3650])),
 (' at', 23, tensor([0.7233, 0.7404])),
 (' the', 5, tensor([0.5626, 0.5693])),
 (' W', 305, tensor([0.0992, 0.1038])),
 ('aver', 9903, tensor([0.7166, 0.6745])),
 ('ley', 607, tensor([0.9400, 0.9403])),
 (' cafe', 16381, tensor([0.5410, 0.5290])),
 (' in', 11, tensor([0.3207, 0.3224])),
 (' which', 61, tensor([0.7555, 0.8088])),
 (' two', 80, tensor([0.6819, 0.7448])),
 (' women', 390, tensor([0.3642, 0.3930])),
 (' were', 58, tensor([0.6122, 0.6101])),
 (' killed', 848, tensor([0.8426, 0.8378])),
 (' by', 30, tensor([0.3655,

### Hacking

In [255]:
model = causal_prior_model
model.eval()
tokenizer = bart_tokenizer
tokenized = tokenizer.encode(
    entfa_sydney["prediction"], 
    return_tensors="pt"
)

for j, t in list(enumerate(tokenized[0]))[:10]:
    print(j, t.item(), tokenizer.decode(t))

0 0 <s>
1 104 S
2 9611 yd
3 2596 ney
4 34  has
5 4760  marked
6 5  the
7 78  first
8 4038  anniversary
9 9  of


In [256]:
input = tokenized[:, :3]
input

tensor([[   0,  104, 9611]])

In [257]:
output = model(input).logits[0]
output.shape

torch.Size([3, 50265])

In [260]:
input = tokenized[:, :3]
output = model(input).logits[0]

for dist in output.softmax(dim=1):
    print(dist.shape)
    print(dist.topk(k=3))

torch.Size([50265])
torch.return_types.topk(
values=tensor([0.5963, 0.2194, 0.0257], grad_fn=<TopkBackward0>),
indices=tensor([36567, 12591, 34479]))
torch.Size([50265])
torch.return_types.topk(
values=tensor([0.4562, 0.3828, 0.0283], grad_fn=<TopkBackward0>),
indices=tensor([36567, 12591,  4450]))
torch.Size([50265])
torch.return_types.topk(
values=tensor([0.0672, 0.0656, 0.0545], grad_fn=<TopkBackward0>),
indices=tensor([  287,  3644, 12921]))


In [259]:
import torch
with torch.no_grad():
    input = tokenized[:, :3]
    output = model(input).logits[0]

    for dist in output.softmax(dim=1):
        print(dist.shape)
        print(dist.topk(k=3))

torch.Size([50265])
torch.return_types.topk(
values=tensor([0.5963, 0.2194, 0.0257]),
indices=tensor([36567, 12591, 34479]))
torch.Size([50265])
torch.return_types.topk(
values=tensor([0.4562, 0.3828, 0.0283]),
indices=tensor([36567, 12591,  4450]))
torch.Size([50265])
torch.return_types.topk(
values=tensor([0.0672, 0.0656, 0.0545]),
indices=tensor([  287,  3644, 12921]))


In [247]:
top = output.softmax(dim=1)[0].topk(k=3)

for prob, tok in zip(top.values, top.indices):
    print(prob.item(), tokenizer.decode(tok), tok.item())

0.5962832570075989 aida 36567
0.21944782137870789  Edge 12591
0.025743484497070312  ESA 34479


In [279]:
from transformers import (
    BartForConditionalGeneration,
)

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [281]:
target = 'Sydney has marked the first anniversary of the siege at the Waverley cafe in which two women were killed by a gunman in the Australian city.'

with torch.no_grad():
    target_tokens = tokenizer(target, return_tensors="pt").input_ids

    logits = model(target_tokens).logits
    probs = logits.squeeze(0).softmax(dim=1)

    preds = []
    for step_probs, target_token in zip(probs, target_tokens[0]):
        preds.append(
            (
                tokenizer.decode(target_token),
                target_token.item(),
                step_probs[target_token].item(),
                step_probs.topk(k=3).indices
            )
        )

preds

[('<s>', 0, 1.609744504094124e-05, tensor([  104,   133, 25533])),
 ('S', 104, 0.994608998298645, tensor([  104,   133, 25533])),
 ('yd', 9611, 0.9998235106468201, tensor([9611, 2981, 4183])),
 ('ney', 2596, 0.9999237060546875, tensor([ 2596, 30915, 17251])),
 (' has', 34, 0.7072009444236755, tensor([   34,  4863, 16381])),
 (' marked', 4760, 0.9997780919075012, tensor([ 4760, 10032, 16293])),
 (' the', 5, 0.9999321699142456, tensor([  5,  63, 133])),
 (' first', 78, 0.9972997307777405, tensor([ 78,  65, 112])),
 (' anniversary', 4038, 0.9999219179153442, tensor([4038,   76,  183])),
 (' of', 9, 0.9997175335884094, tensor([ 9, 19,  6])),
 (' the', 5, 0.999014139175415, tensor([ 5, 10, 41])),
 (' siege', 19951, 0.9994810223579407, tensor([19951,   305, 16381])),
 (' at', 23, 0.9993714690208435, tensor([  23, 3750,   11])),
 (' the', 5, 0.9999973773956299, tensor([  5,  10, 305])),
 (' W', 305, 0.9999921321868896, tensor([  305,   580, 16381])),
 ('aver', 9903, 0.9999996423721313, tensor

[('<s>', 0, 2.1549179541474617e-12),
 ('S', 104, 1.780824277375359e-06),
 ('yd', 9611, 2.7244109332968947e-06),
 ('ney', 2596, 3.707136642105979e-08),
 (' has', 34, 0.027413560077548027),
 (' marked', 4760, 1.1275883480266202e-06),
 (' the', 5, 0.00018941506277769804),
 (' first', 78, 0.0023676673881709576),
 (' anniversary', 4038, 5.814571068185614e-06),
 (' of', 9, 0.0017079596873372793),
 (' the', 5, 0.003059038193896413),
 (' siege', 19951, 3.6858385787930104e-10),
 (' at', 23, 0.0022077469620853662),
 (' the', 5, 0.0014361185021698475),
 (' W', 305, 0.002821172820404172),
 ('aver', 9903, 3.773652963445784e-07),
 ('ley', 607, 7.092088338822577e-08),
 (' cafe', 16381, 1.8298884185696807e-07),
 (' in', 11, 0.03579285740852356),
 (' which', 61, 2.48848186856776e-06),
 (' two', 80, 4.862476998823695e-05),
 (' women', 390, 0.00013294632663019001),
 (' were', 58, 0.00033202560734935105),
 (' killed', 848, 4.035213407860283e-08),
 (' by', 30, 4.279447966837324e-05),
 (' a', 10, 0.00526131